In [1]:
%matplotlib inline

import os
import sys
import datacube
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

sys.path.append("Scripts")
from dea_plotting import display_map
from dea_datahandling import wofs_fuser
from dea_plotting import rgb
from dea_datahandling import load_ard
from dea_bandindices import calculate_indices

In [4]:
dc = datacube.Datacube(app="Regression_test")

In [9]:
# setting some appropriate variables

lat = (-34,-34.5)
lon = (145,145.5)
time = ("2015-01-17","2015-02-22")

In [10]:
# querying for some data to test the regression against

query = {'x':lon,
        'y':lat,
        'time':time,
        'output_crs':'EPSG:3577',
        'resolution':(-25,25),
        'measurements':['nbart_swir_1','nbart_red','nbart_blue','nbart_green','nbart_swir_2','nbart_panchromatic','nbart_nir'],
        'group_by':'solar_day'
        }
regress = dc.load(product='ga_ls8c_ard_3',**query)

In [11]:
# NDVI indexing for use in the regreswsion comparison

regress['NDVI'] = (regress.nbart_nir-regress.nbart_red)/(regress.nbart_nir+regress.nbart_red)

In [12]:
# normalising the swir_1 (band 6) as an index relative to its maximum and minimum
swir_1_max = regress.nbart_swir_1.max()
swir_1_min = regress.nbart_swir_1.min()

regress["NSWIR"] = (regress.nbart_swir_1-swir_1_min)/(swir_1_max-swir_1_min)

In [13]:
print(regress)

<xarray.Dataset>
Dimensions:             (time: 2, x: 2056, y: 2407)
Coordinates:
  * time                (time) datetime64[ns] 2015-01-24T00:08:23.004012 2015-02-09T00:08:19.307218
  * y                   (y) float64 -3.771e+06 -3.772e+06 ... -3.832e+06
  * x                   (x) float64 1.187e+06 1.187e+06 ... 1.238e+06 1.238e+06
Data variables:
    nbart_swir_1        (time, y, x) int16 2315 2271 2253 ... 2644 2588 2536
    nbart_red           (time, y, x) int16 7589 7556 7547 ... 1373 1367 1390
    nbart_blue          (time, y, x) int16 7635 7577 7568 7566 ... 719 705 701
    nbart_green         (time, y, x) int16 7578 7566 7555 ... 1096 1067 1027
    nbart_swir_2        (time, y, x) int16 2410 2393 2377 ... 1944 1949 1997
    nbart_panchromatic  (time, y, x) int16 7576 7555 7533 ... 1111 1051 1029
    nbart_nir           (time, y, x) int16 7399 7354 7343 ... 2065 2072 2036
    NDVI                (time, y, x) float64 -0.01268 -0.01355 ... 0.205 0.1886
    NSWIR               (tim

In [14]:
regress.NSWIR

<xarray.DataArray 'NSWIR' (time: 2, y: 2407, x: 2056)>
array([[[0.33546 , 0.329081, ..., 0.73659 , 0.717309],
        [0.342273, 0.335315, ..., 0.744129, 0.720644],
        ...,
        [0.300667, 0.301827, ..., 0.893302, 0.893302],
        [0.298637, 0.302406, ..., 0.900551, 0.885474]],

       [[0.484923, 0.475065, ..., 0.670049, 0.669035],
        [0.491157, 0.487098, ..., 0.688895, 0.683242],
        ...,
        [0.639461, 0.649464, ..., 0.391563, 0.391563],
        [0.635981, 0.64091 , ..., 0.375036, 0.367498]]])
Coordinates:
  * time     (time) datetime64[ns] 2015-01-24T00:08:23.004012 2015-02-09T00:08:19.307218
  * y        (y) float64 -3.771e+06 -3.772e+06 ... -3.832e+06 -3.832e+06
  * x        (x) float64 1.187e+06 1.187e+06 1.187e+06 ... 1.238e+06 1.238e+06